# Azure AI Agent service - Multi Agent team

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

## Description

This sample demonstrates how to use multiple agents with `AgentTeam` and traces.

The team consists of:
- **One leader agent** – automatically created by `AgentTeam` from the configuration in [`utils/agent_team_config.yaml`](../utils/agent_team_config.yaml)
- **Two worker agents** – `Coder` and `Reviewer`, defined in the code below


In [1]:
import json
import os
import sys
import time

from typing import Optional, Set
from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential
from utils.agent_team import AgentTeam, _create_task, AgentTask
from utils.agent_trace_configurator import AgentTraceConfigurator
from azure.ai.agents.models import FunctionTool, ToolSet

from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()

True

## Create connection to AI Foundry Project & instantiate a project client

In [2]:
endpoint = os.getenv("PROJECT_ENDPOINT")
print(f"Using endpoint: {endpoint}")
credential = DefaultAzureCredential()

agents_client = AgentsClient(
    endpoint=endpoint,
    credential=credential,
)

Using endpoint: https://aq-ai-foundry-sweden-central.services.ai.azure.com/api/projects/firstProject


In [3]:
agents_client.enable_auto_function_calls({_create_task})

## This section demonstrates how to multiple agents using AgentTeam with traces.

In [4]:
def create_task(team_name: str, recipient: str, request: str, requestor: str) -> str:
    """
    Requests another agent in the team to complete a task.

    :param team_name (str): The name of the team.
    :param recipient (str): The name of the agent that is being requested to complete the task.
    :param request (str): A description of the to complete. This can also be a question.
    :param requestor (str): The name of the agent who is requesting the task.
    :return: True if the task was successfully received, False otherwise.
    :rtype: str
    """
    task = AgentTask(recipient=recipient, task_description=request, requestor=requestor)
    team: Optional[AgentTeam] = None
    try:
        team = AgentTeam.get_team(team_name)
    except:
        pass
    if team is not None:
        team.add_task(task)
        return "True"
    return "False"

In [5]:
# Any additional functions that might be used by the agents:
agent_team_default_functions: Set = {
    create_task,
}

default_function_tool = FunctionTool(functions=agent_team_default_functions)

agents_client.enable_auto_function_calls({create_task})

In [6]:
model_deployment_name = "gpt-4.1-mini"

if model_deployment_name is not None:
    AgentTraceConfigurator(agents_client=agents_client).setup_tracing()
    with agents_client:
        agent_team = AgentTeam("test_team", agents_client=agents_client)
        toolset = ToolSet()
        toolset.add(default_function_tool)
        agent_team.set_team_leader(
            model=model_deployment_name,
            name="TeamLeader",
            instructions="You are an agent named 'TeamLeader'. You are a leader of a team of agents. The name of your team is 'test_team'."
            "You are an agent that is responsible for receiving requests from user and utilizing a team of agents to complete the task. "
            "When you are passed a request, the only thing you will do is evaluate which team member should do which task next to complete the request. "
            "You will use the provided create_task function to create a task for the agent that is best suited for handling the task next. "
            "You will respond with the description of who you assigned the task and why. When you think that the original user request is "
            "processed completely utilizing all the talent available in the team, you do not have to create anymore tasks. "
            "Using the skills of all the team members when applicable is highly valued. "
            "Do not create parallel tasks. "
            "Here are the other agents in your team: "
            "- Coder: You are software engineer who writes great code. Your name is Coder. "
            "- Reviewer: You are software engineer who reviews code. Your name is Reviewer.",
            toolset=toolset,
        )
        agent_team.add_agent(
            model=model_deployment_name,
            name="Coder",
            instructions="You are software engineer who writes great code. Your name is Coder.",
        )
        agent_team.add_agent(
            model=model_deployment_name,
            name="Reviewer",
            instructions="You are software engineer who reviews code. Your name is Reviewer.",
        )
        agent_team.assemble_team()

        print("A team of agents specialized in software engineering is available for requests.")
        while True:
            user_input = input("Input (type 'quit' or 'exit' to exit): ")
            if user_input.lower() == "quit":
                break
            elif user_input.lower() == "exit":
                break
            agent_team.process_request(request=user_input)

        agent_team.dismantle_team()
else:
    print("Error: Please define the environment variable MODEL_DEPLOYMENT_NAME.")


Select a tracing option:
1. Enable Azure Monitor tracing
2. Enable console tracing without enabling gen_ai agent traces
3. Enable console tracing with gen_ai agent traces
4. Do not enable traces
A team of agents specialized in software engineering is available for requests.
Created thread with ID: thread_BMfn2gDotiMOGMcrfrFKKdbi
Starting task for agent 'TeamLeader'. Requestor: 'user'. Task description: 'Please create a task for agent in the team that is best suited to next process the following request. 
Use the _create_task function available for you to create the task. The request is: 
1
'.
Created message with ID: msg_MLmYDLVCe64WWHIiOnHwM0Te for task in thread thread_BMfn2gDotiMOGMcrfrFKKdbi
Created and processed run for agent 'TeamLeader', run ID: run_q4d0xJt95nLFubRVmDjKN36q
Agent 'TeamLeader' completed task. Outcome: I have assigned the task to Coder because the request "1" seems to be something that requires processing or coding. Coder is best suited to handle this next step.
S